In [0]:
import pandas as pd
import numpy as np

In [5]:
J1939 = pd.read_csv('J1939Faults.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
J1939.shape

(1127339, 20)

In [0]:
service = pd.read_excel('Service Fault Codes_1_0_0_167.xlsx')

In [9]:
service.shape

(7124, 14)

In [0]:
vehicle = pd.read_csv('VehicleDiagnosticOnboardData.csv')

In [11]:
vehicle.shape

(12821626, 4)

In [12]:
vehicle.head(20)

,Id,Name,Value,FaultId
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1
5,6,AcceleratorPedal,0,1
6,7,IntakeManifoldTemperature,78.8,1
7,8,FuelRate,0,1
8,9,FuelLtd,12300.907429328,1
9,10,EngineRpm,0,1


In [20]:
J1939.head()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000


In [19]:
J1939.EquipmentID.value_counts()

1641         17216
1646         15174
1618         14774
1605         14748
1606         14654
             ...  
2208             1
105438645        1
105435406        1
105301976        1
105381862        1
Name: EquipmentID, Length: 1836, dtype: int64

In [17]:
service.head(20)

,Published in CES 14602,Cummins Fault Code,Revision,PID,SID,MID,J1587 FMI,SPN,J1939 FMI,J2012 Pcode,Lamp Color,Lamp Device,Cummins Description,Algorithm Description
0,Y,111,167,Not Mapped,254,0,12,629,12,P0606,Red,Stop / Shutdown,Engine Control Module Critical Internal Failur...,Error internal to the ECM related to memory ha...
1,Y,112,167,Not Mapped,20,128,7,635,7,Not Mapped,Red,Stop / Shutdown,Engine Timing Actuator Driver Circuit - Mechan...,Mechanical failure in the engine timing actuat...
2,Y,113,167,Not Mapped,20,128,3,635,3,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,High signal voltage detected at the engine tim...
3,Y,114,167,Not Mapped,20,128,4,635,4,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,Low voltage detected at the engine timing actu...
4,Y,115,167,190,Not Mapped,Not Mapped,2,612,2,P0008,Red,Stop / Shutdown,Engine Magnetic Speed/Position Lost Both of Tw...,The ECM has detected that the primary and back...
5,Y,116,167,156,Not Mapped,Not Mapped,3,156,3,Not Mapped,Red,Stop / Shutdown,Injector Timing Rail 1 Fuel Pressure Sensor Ci...,High signal voltage detected at the Injector T...
6,Y,117,167,156,Not Mapped,Not Mapped,4,156,4,Not Mapped,Red,Stop / Shutdown,Injector Timing Rail 1 Fuel Pressure Sensor Ci...,Low signal voltage detected at the Injector Ti...
7,Y,118,167,135,Not Mapped,Not Mapped,3,135,3,Not Mapped,Amber,Warning,Fuel Delivery Pressure 1 Sensor Circuit - Volt...,High signal voltage detected at the fuel deliv...
8,Y,119,167,135,Not Mapped,Not Mapped,4,135,4,Not Mapped,Amber,Warning,Fuel Delivery Pressure 1 Sensor Circuit - Volt...,Low signal voltage detected at the fuel delive...
9,Y,121,167,190,Not Mapped,Not Mapped,10,190,10,Not Mapped,Amber,Warning,Engine Magnetic Crankshaft Speed/Position Lost...,The ECM has lost one of the two speed/position...


In [0]:
merged_df = pd.merge(J1939, vehicle, left_on='RecordID', right_on='FaultId', how='left')

In [27]:
merged_df.tail()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,Id,Name,Value,FaultId
12152079,1178755,93949768,2019-07-13 06:28:08.000,High Voltage (Fuel Level),NaN,NaN,NaN,CECU3B-NAMUX4,PACCR,49,96,3,True,126,NaN,1997,105317134,36.194861,-83.174768,2019-07-13 06:28:44.000,12187409,ParkingBrake,False,1178755
12152080,1178755,93949768,2019-07-13 06:28:08.000,High Voltage (Fuel Level),NaN,NaN,NaN,CECU3B-NAMUX4,PACCR,49,96,3,True,126,NaN,1997,105317134,36.194861,-83.174768,2019-07-13 06:28:44.000,12187410,DistanceLtd,317859.4,1178755
12152081,1178756,93949844,2019-07-13 06:30:38.000,Low (Severity Low) Engine Coolant Level,NaN,05317106*05002234*050515205406*09400034*G1*BDR*,79845329,6X1u13D1500000000,CMMNS,0,111,17,False,126,NaN,1809,105416085,30.292638,-91.914166,2019-07-13 06:30:34.000,12187411,LampStatus,17407,1178756
12152082,1178757,93949882,2019-07-13 06:31:44.000,Low (Severity Medium) Engine Coolant Level,NaN,04358814*06011419*051718174436*09401683*G1*BDR*,NaN,NaN,NaN,0,111,18,False,40,NaN,1864,105420775,36.194768,-83.174768,2019-07-13 06:31:39.000,12187412,LampStatus,1023,1178757
12152083,1178758,93949927,2019-07-13 06:32:14.000,NaN,NaN,04358814*06011419*051718174436*09401683*G1*BDR*,NaN,NaN,NaN,0,4339,7,False,2,NaN,1864,105420775,36.194767,NaN,NaN,12187413,LampStatus,1023,1178758


In [32]:
failure_error_2 = merged_df.spn.isin([5246])
failure_error_2


0           False
1           False
2           False
3           False
4           False
            ...  
12152079    False
12152080    False
12152081    False
12152082    False
12152083    False
Name: spn, Length: 12152084, dtype: bool

In [37]:
failure_error = merged_df.spn.isin([1569]) & merged_df.fmi.isin([31])
failure_error


0           False
1           False
2           False
3           False
4           False
            ...  
12152079    False
12152080    False
12152081    False
12152082    False
12152083    False
Length: 12152084, dtype: bool

In [38]:
merged_df[failure_error]

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp,Id,Name,Value,FaultId
369,41,990856,2015-02-21 12:06:22.000,Condition Exists Engine Protection Torque Derate,NaN,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,31,True,5,NaN,1721,105439606,39.051805,-84.560509,2015-02-21 12:06:57.000,370,Throttle,0,41
370,41,990856,2015-02-21 12:06:22.000,Condition Exists Engine Protection Torque Derate,NaN,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,31,True,5,NaN,1721,105439606,39.051805,-84.560509,2015-02-21 12:06:57.000,371,IgnStatus,True,41
371,41,990856,2015-02-21 12:06:22.000,Condition Exists Engine Protection Torque Derate,NaN,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,31,True,5,NaN,1721,105439606,39.051805,-84.560509,2015-02-21 12:06:57.000,372,EngineOilPressure,35.96,41
372,41,990856,2015-02-21 12:06:22.000,Condition Exists Engine Protection Torque Derate,NaN,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,31,True,5,NaN,1721,105439606,39.051805,-84.560509,2015-02-21 12:06:57.000,373,EngineOilTemperature,220.4375,41
373,41,990856,2015-02-21 12:06:22.000,Condition Exists Engine Protection Torque Derate,NaN,05317106*04047493*092613211021*09300006*G1*BDR*,79723635,6X1u13D1500000000,CMMNS,0,1569,31,True,5,NaN,1721,105439606,39.051805,-84.560509,2015-02-21 12:06:57.000,374,FuelTemperature,32,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12149482,1178523,93859287,2019-07-12 10:34:08.000,Condition Exists Engine Protection Torque Derate,NaN,04358814*06030804*100515184137*09400147*G1*BDR*,79905485,6X1u13D1500000000,CMMNS,0,1569,31,True,31,NaN,1908,105340150,35.720925,-81.181990,2019-07-12 10:34:44.000,12184812,CruiseControlActive,False,1178523
12149483,1178523,93859287,2019-07-12 10:34:08.000,Condition Exists Engine Protection Torque Derate,NaN,04358814*06030804*100515184137*09400147*G1*BDR*,79905485,6X1u13D1500000000,CMMNS,0,1569,31,True,31,NaN,1908,105340150,35.720925,-81.181990,2019-07-12 10:34:44.000,12184813,EngineCoolantTemperature,185,1178523
12149484,1178523,93859287,2019-07-12 10:34:08.000,Condition Exists Engine Protection Torque Derate,NaN,04358814*06030804*100515184137*09400147*G1*BDR*,79905485,6X1u13D1500000000,CMMNS,0,1569,31,True,31,NaN,1908,105340150,35.720925,-81.181990,2019-07-12 10:34:44.000,12184814,ParkingBrake,False,1178523
12149485,1178523,93859287,2019-07-12 10:34:08.000,Condition Exists Engine Protection Torque Derate,NaN,04358814*06030804*100515184137*09400147*G1*BDR*,79905485,6X1u13D1500000000,CMMNS,0,1569,31,True,31,NaN,1908,105340150,35.720925,-81.181990,2019-07-12 10:34:44.000,12184815,DistanceLtd,352550.2,1178523
